# Clean EPA FLIGHT Excel (.xls) export
This notebook reads the multi-tab FLIGHT export (one tab per year), strips the metadata rows, standardizes columns, coerces types, and exports a single cleaned CSV.

In [5]:
from pathlib import Path
import re
import numpy as np
import pandas as pd

In [6]:
RAW_PATH = Path("../data/flight.xls")
OUT_VA   = Path("../data/flight_cleaned_va_all_years.csv")

## Helpers: read each sheet (find header row) + cleaning rules

In [7]:
def read_flight_sheet(xl_path, sheet_name):
    raw = pd.read_excel(xl_path, sheet_name=sheet_name, header=None, engine="xlrd")

    header_row = None
    for i in range(min(50, len(raw))):
        row = raw.iloc[i].astype(str).str.upper()
        if row.str.contains("REPORTING YEAR").any():
            header_row = i
            break
    if header_row is None:
        raise ValueError(f"Could not find header in sheet {sheet_name}")

    headers = raw.iloc[header_row].tolist()
    df = raw.iloc[header_row + 1 :].copy()
    df.columns = headers
    df = df.dropna(how="all")
    return df

def normalize_columns(cols):
    out = []
    for c in cols:
        c = str(c).strip().lower()
        c = c.replace("(", "").replace(")", "")
        c = re.sub(r"[^a-z0-9]+", "_", c)
        c = re.sub(r"_+", "_", c).strip("_")
        out.append(c)
    return out

def clean_flight_df(df):
    df = df.copy()
    df.columns = normalize_columns(df.columns)

    # reporting year
    df["reporting_year"] = pd.to_numeric(df.get("reporting_year"), errors="coerce").astype("Int64")
    df = df.dropna(subset=["reporting_year"])
    df["reporting_year"] = df["reporting_year"].astype(int)

    # trim text columns
    for col in df.select_dtypes(include=["object", "string"]).columns:
        df[col] = (
            df[col]
            .astype(str)
            .replace({"nan": np.nan, "None": np.nan})
            .str.strip()
            .str.replace(r"\s+", " ", regex=True)
        )

    # ids + numerics
    if "ghgrp_id" in df.columns:
        df["ghgrp_id"] = pd.to_numeric(df["ghgrp_id"], errors="coerce").astype("Int64")

    for col in ["latitude", "longitude"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")

    qcol = "ghg_quantity_metric_tons_co2e"
    if qcol in df.columns:
        df[qcol] = pd.to_numeric(df[qcol], errors="coerce")
        df["ghg_is_negative"] = df[qcol] < 0
        df.loc[df["ghg_is_negative"], qcol] = np.nan

    # zip/state normalization
    if "zip_code" in df.columns:
        df["zip_code"] = df["zip_code"].apply(
            lambda x: str(int(x)).zfill(5)
            if pd.notna(x) and str(x).strip().replace(".0", "").isdigit()
            else (str(x).strip() if pd.notna(x) else np.nan)
        ).replace({"": np.nan, "nan": np.nan})

    if "state" in df.columns:
        df["state"] = df["state"].str.upper().str.strip()
        df.loc[df["state"].isin(["NAN", "NONE", ""]), "state"] = np.nan

    # pretty casing (optional)
    if "county_name" in df.columns:
        df["county_name"] = df["county_name"].where(df["county_name"].isna(), df["county_name"].str.title())
    if "city_name" in df.columns:
        df["city_name"] = df["city_name"].where(df["city_name"].isna(), df["city_name"].str.title())

    return df

## Load all years and export

In [8]:
xl = pd.ExcelFile(RAW_PATH)
va_parts = []

for sh in xl.sheet_names:
    df_sh = read_flight_sheet(RAW_PATH, sh)
    df_sh = clean_flight_df(df_sh)

    # Filter VA only early (faster + smaller)
    df_sh = df_sh.loc[df_sh["state"] == "VA"].copy()

    df_sh["source_sheet"] = sh
    va_parts.append(df_sh)

va = pd.concat(va_parts, ignore_index=True)

OUT_VA.parent.mkdir(parents=True, exist_ok=True)
va.to_csv(OUT_VA, index=False)

print("Saved:", OUT_VA.resolve())
print("Rows:", len(va), "| Years:", sorted(va["reporting_year"].unique()))

Saved: C:\Users\blake\va-ghg-dashboard\data\flight_cleaned_va_all_years.csv
Rows: 2056 | Years: [np.int64(2010), np.int64(2011), np.int64(2012), np.int64(2013), np.int64(2014), np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023)]
